# Đồ án 4: Xử lí ngôn ngữ tự nhiên / Mô hình hồi quy tối tiểu

## Thông tin

- **Họ và tên:** Bùi Vũ Hiếu Phụng
- **MSSV:** 18127185
- **Lớp:** Toán ứng dụng và Thống kê - MTH00051 @ 18CLC4
- **Github:** [@alecmatts](https://github.com/alecmatts)

## Import thư viện

In [ ]:
# DON'T CHANGE this part: import libraries
import numpy as np
import scipy
import json
from nltk.stem import PorterStemmer 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import re
import itertools

## Xử lý dữ liệu dạng văn bản

- **Khởi tạo các global object cần thiết cho việc tiền xử lí**. Bao gồm:
    - vocab: Bộ từ điển của chương trình, bao gồm các từ mới xuất hiện trong quá trình training. Ở đây được khởi tạo là một `list`.
    - stemmer: Cổng stem để biến đổi một từ về từ gốc (root form), đưa các biến thể của từ về một từ duy nhất. Khởi tạo bằng object `nltk.stem.PorterStemmer` được khai báo ở phần import.
    - stopwords: Bộ những từ phổ biến nhưng thừa thãi, không liên quan đến ngữ nghĩa của văn bản. Các từ này dễ gây nhiễu data, cản trở việc xử lí/phân loại văn bản nên cần được lược bỏ. Để lấy các từ này, ta dùng `nltk.corpus.stopwords`.

In [ ]:
vocab = []
stemmer = PorterStemmer()
stopwords = set(stopwords.words('english'))

- **Xử lí chuỗi số (Chuyển các chuỗi số thành 'num')**
    - Ban đầu em sử dụng RegEx để xác định các chuỗi số (Reference: [link](https://stackoverflow.com/questions/2811031/decimal-or-numeric-values-in-regular-expression-validation)) nhưng không thu được kết quả như mong đợi vì thiếu kiến thức về RegEx cũng như có nhiều trường hợp không thể bao quát hết được.
    - Dưới đây, em chọn ép kiểu và `try/except` để xác định các chuỗi số, dễ đọc và dễ tiếp cận hơn. (Reference: [link](https://stackoverflow.com/questions/1265665/how-can-i-check-if-a-string-represents-an-int-without-using-try-except/1267145#1267145)). Dùng `try/except` để kiểm tra việc ép kiểu có hợp lệ không. Nếu:
        - Có thể ép kiểu mà không báo lỗi thì chuỗi đó là số nguyên/thực. Ta trả về 'num'.
        - Các trường hợp khác sẽ trả về chuỗi đó mà không có thay đổi gì.

In [ ]:
def numify(string):
    # Khởi tạo biến để kiểm tra chuỗi số học hay không
    isNumeric = False

    # Kiểm tra tính hợp lệ của việc ép kiểu để xác định chuỗi số 
    try:
        int(string) # Số nguyên
        isNumeric = True
    except ValueError:
        pass

    try:
        float(string) # Số thực
        isNumeric = True
    except ValueError: 
        pass
    
    # Các trường hợp trả về
    return 'num' if isNumeric else string

- **Tiền xử lí văn bản.**
    - Ở đây cần phải đọc 2 bộ dữ liệu: train và valid/test. Nên có thêm một tham số đầu vào `train` nhận giá trị `True/False` để xác định loại dữ liệu.
    - Bao gồm các công việc chung sau:
        - Chuyển tất cả thành chữ thường. Dùng phương thức `string.lower()` của `python`.
        - Tách từ. Dùng `nltk.tokenize.word_tokenize` từ thư viện `nltk` hỗ trợ cho việc xử lí ngôn ngữ tự nhiên.
        - Loại bỏ stopwords. Duyệt qua từng phần từ của list từ bằng list comprehension, giữ lại các từ không thuộc `stopword`.
        - Chuyển thành từ gốc. Dùng cổng `stemmer` đã khởi tạo ở đầu chương trình.
        - Chuyển số thành 'num'. Sử dụng hàm `numify` đã cài đặt ở trên
        - Đối với từng loại dữ liệu: Tương tác với `list` bình thường (dùng `append`)
            - Bộ train: Thêm từ vào kết quả tiền xử lí của văn bản và thêm từ mới vào bộ từ điển của chương trình
            - Bộ test: Thêm từ tồn tại trong bộ từ điển vào kết quả tiền xử lí của văn bản. Các từ chưa từng xuất hiện trong quá trình học sẽ được chuyển thành chuỗi 'unk'  

In [ ]:
def preprocess(text, train):
    # Chuyển tất cả thành chữ thường
    text = text.lower()
    
    # Tách từ thành list
    tokens = word_tokenize(text)

    # Loại bỏ stopwords
    tokens = [t for t in tokens if not t in stopwords]

    # Thêm từ vào kết quả tiền xử lí
    result = []
    for word in tokens:        
        word = stemmer.stem(word) # Chuyển thành từ gốc
        word = numify(word)       # Chuyển số thành 'num'

        # Các điều kiện và hành vi với từng loại dữ liệu
        if train:
            result.append(word)
            if word not in vocab:
                vocab.append(word)
        else:
            if word not in vocab:
                result.append('unk')
            else:
                result.append(word)

    return result

- **Đọc dữ liệu từ đường dẫn đến file**
    - Mở và đọc file bằng `open()` và `json.load()`. Ta chỉ phân tích đánh giá và số điểm mà người dùng đưa ra nên ta chỉ cần quan tâm đến hai trường dữ liệu: `reviewText` và `overall` - trích xuất bằng cú pháp `<khối dữ liệu>['<trường dữ liệu>']` 
    - Phần văn bản lưu kết quả tiền xử lí của văn bản thông qua chạy hàm `preprocess` đã cài ở trên. Phần điểm (còn được coi là nhãn của văn bản) là một `np.array` chứa các số tương đương với số điểm mà người dùng chấm.
    - Lưu chúng ở hai `list` khác nhau nhưng dễ dàng truy xuất đồng thời thông qua `index`. VD: ở index `i` ta có kết quả tiền xử lí của văn bản thứ `i` và nhãn của văn bản thứ `i`

In [ ]:
def read(path, train):
    # Mở và đọc file
    with open(path, 'r') as f:
        data = json.load(f)
    
    # Lưu kết quả tiền xử lí và nhãn văn bản như đã nói ở trên
    pp_documents = []
    labels = []
    for block in data:
        pp_documents.append(preprocess(block['reviewText'], train))
        labels.append(int(block['overall']))
    
    return pp_documents, np.array(labels)

- **Tạo histogram vector**
    - Khởi tạo một vector toàn số 0 với số chiều bằng độ dài của bộ từ điển. `numpy` có phương thức `np.zeros(shape)` hỗ trợ công việc này.
    - Đếm số lần xuất hiện của mỗi từ thuộc bộ từ điển trong đoạn văn hiện tại rồi cập nhật phần tử vector tại vị trí đó. Vì kết quả sau khi tiền xử lí văn bản là một `list` các từ, để thực hiện đếm ta chỉ cần gọi `list.count()` với tham số đầu vào là phần từ cần đếm. Các từ 'unk' sẽ không được đếm.
    - Áp dụng công thức a / (1.T @ a) để tạo vector tần suất từ.
        - Tạo vector 1 bằng `np.ones(shape)`.
        - Dùng toán tử `@` để thực hiện phép nhân.
        - Có thể dùng toán từ `/` hoặc dùng `np.divide` để tính toán phép chia ma trận.

In [ ]:
def histogram_vector(document):
    # Khởi tạo vector 0
    word_count_vector = np.zeros(len(vocab))

    # Đếm số lần xuất hiện của mỗi từ
    for i, v in enumerate(vocab):
        word_count_vector[i] = document.count(v)
    
    # Áp dụng công thức để tạo vector tần suất từ
    return np.divide(word_count_vector, np.ones(len(vocab)).T @ word_count_vector)

- **Tạo ma trận histogram (document-term matrix)**
    - Mỗi văn bản qua hàm `histogram_vector` sẽ tạo thành 1 histogram vector. Xếp các vector này theo hàng ta được ma trận histogram
    - Để xếp các vector theo dòng, dùng hàm `np.vstack()` rồi truyền vào đó `list` các mảng muốn xếp.

In [ ]:
def document_term_matrix(documents):
    # Xếp các histogram vector thành ma trận
    return np.vstack([histogram_vector(doc) for doc in documents])

## Sử dụng mô hình hồi quy tuyến tính dùng bình phương tối tiểu

- **Xử lí nhãn văn bản**
    - Do em chọn mô hình M2 nên cần dự đoán nhãn theo trường overall từ 1-5. Với mô hình này, kết quả của hồi quy tuyến tính một vector `y` 5 chiều, mỗi phần tử tại `i` của vector là xác suất để `overall = i + 1`.
    - Nhãn văn bản có xác suất là 100% nên tại nếu `overall = i` thì phần tử có index `i - 1` sẽ là `1`, còn lại là `0` vì sẽ xác suất overall rơi vào đó là 0%. VD: `overall = 5` thì `y = [0 0 0 0 1]`.
    - Để tạo được vector này, ta dùng `np.identity(5)` để tạo ma trận đơn vị 5x5. Tại __dòng__ thứ `i` của ma trận này, phần tử ở __cột__ thứ i là 1.
    - Tham số truyền vào là ma trận cột các nhãn văn bản nên ta xử lí tương tự cho từng nhãn rồi `np.vstack` chúng theo dòng để có kết quả mong muốn. 

In [ ]:
def getYs(ys):
    return np.vstack([np.identity(5)[rate - 1] for rate in ys])

- **Lấy các tham số A, b cho hàm Linear Regression**
    - Đối với ma trận A: Ghép ma trận cột 1 với ma trận tần suất từ `xs`. Dùng `np.concatenate` trên `axis=1` để ghép.
    - Đối với ma trận b: Xử lí nhãn văn bản `ys` bằng hàm `getYs()` đã cài đặt ở trên.

In [ ]:
def getParameters(xs, ys):
    A = np.concatenate((np.ones((len(xs), 1)), xs), axis=1)
    b = getYs(ys)
    return A, b

- **Xây dựng mô hình bằng phương pháp hồi quy tuyến tính**
    - Áp dụng công thức x_hat = (A.T @ A) ^ -1 @ A.T @ b, trong đó: giả nghịch đảo(A) = (A.T @ A) ^ -1 @ A.T
    - Giả nghịch đảo có thể được tính bằng `np.linalg.pinv()` và toán tử `@` để thực hiện phép nhân.

In [ ]:
def LinearRegression(A, b):
    return np.linalg.pinv(A) @ b

- **Phân loại (dự đoán số điểm) tập dữ liệu bằng mô hình dựng được**
    - Áp dụng `Ax = y`, ta thu được tập `y`. 
    - Như đã nói ở trên, vector nhãn được thể hiện dưới dạng xác suất rơi vào từng loại điểm. Vậy ta phải chuyển `y` thu được về vector xác suất thông qua hàm `scipy.special.softmax()`.
    - Index có xác suất cao nhất là nhãn được dự đoán của văn bản. Dùng `np.argmax()` để lấy ra index của phần tử lớn nhất.
    - Mảng nhãn văn bản có index từ 0-4 nhưng số điểm của người dùng chấm thuộc khoảng 1-5 bên sau khi lấy ra index của phần tử lớn nhất ta phải cộng thêm 1

In [ ]:
def classify(A_test, model):
    return np.argmax(scipy.special.softmax(A_test @ model, axis=1), axis=1) + 1

## Sử dụng độ chính xác để đánh giá mô hình

- **Tính độ chính xác qua nhãn phân loại được và nhãn thực**
    - Độ chính xác được tính theo công thức: Số nhãn đúng / Tổng số nhãn
    - Để đếm số nhãn đúng, tạo một ma trận tạm với điều kiện bằng với nhãn thực sự gồm các giá trị `True/False`. Sau đó dùng `np.count_nonzero()` để đếm số giá trị `True` trong đó.

In [ ]:
def accuracy(predict, labels_test):
    return np.count_nonzero(predict == labels_test) / len(labels_test)

## Testing

### Nhận các tham số

In [ ]:
# DON'T CHANGE this part: read data path
train_set_path, valid_set_path, random_number = input().split()

### Đọc và tiền xử lí bộ train và test

In [ ]:
docs_train, labels_train = read(train_set_path, True)
docs_test, labels_test = read(valid_set_path, False)

### In ra kết quả tiền xử lí thứ `random_number` trong bộ valid/test

In [ ]:
print(docs_test[int(random_number)])

### Tạo các ma trận đầu vào cho việc dựng mô hình và kiểm mô hình

In [ ]:
matrix_train = document_term_matrix(docs_train)
A_train, b_train = getParameters(matrix_train, labels_train)

matrix_test = document_term_matrix(docs_test)
A_test, b_test = getParameters(matrix_test, labels_test)

### Xây dựng mô hình

In [ ]:
model = LinearRegression(A_train, b_train)

### Từ model đã có, phân loại văn bản trong tập train/valid

In [ ]:
predict = classify(A_test, model)

### Tính độ chính xác

In [ ]:
acc = accuracy(predict, labels_test)

### In ra màn hình loại mô hình (M1/M2) và độ chính xác của nó

In [ ]:
print('M2 - ' + str(acc))